In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from  nltk import FreqDist
from nltk.corpus import stopwords
import string

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('jobs.db')
c = conn.cursor()

In [5]:
c.execute("""SELECT * FROM listings""")

rows = c.fetchall()

print(len(rows))


980


In [6]:
rows[0]

(0,
 'Data Scientist',
 'Datadog',
 'New York, NY',
 "About DatadogWe're on a mission to build the best platform in the world for engineers to understand and scale their systems, applications, and teams. We operate at high scale—trillions of data points per day—providing always-on alerting, metrics visualization, logs, and application tracing for tens of thousands of companies. Our engineering culture values pragmatism, honesty, and simplicity to solve hard problems the right way.The OpportunityYou will have a fantastic team of data engineers to support you, a collaborative environment to encourage your work, and the best technologies for performing data science at high scale in your toolkit.You Will Present the latest academic research papers to your team. Research and benchmark the latest algorithms that can be used for our particular use-cases. Apply machine learning algorithms and statistical techniques to build new product features. Deploy a new feature to production, instantly af